# Question 8 part 2
## Using Clustering Techniques to label based on spending habits

Some exploritory analysis was done on this new dataset and picked the top five countries that have spent the most in scotland from `2013-2019` see [here..](../data/extra_cleaning.ipynb) for more details into the exact methods these countries we're selected.

Top Five:
- `USA` `Germany` `Australia` `France` `Canada`

## Load Cleaned Data & Libraries

In [1]:
# Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#set plot sizes
sns.set_context('poster', font_scale=0.4)
plt.rcParams['figure.figsize'] = [5, 3]
plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 10

# Cleaned Data
usa = pd.read_csv('../data/clean_data/countries/usa.csv')
germany = pd.read_csv('../data/clean_data/countries/germany.csv')
australia = pd.read_csv('../data/clean_data/countries/australia.csv')
france = pd.read_csv('../data/clean_data/countries/france.csv')
canada = pd.read_csv('../data/clean_data/countries/canada.csv')

### K-Means Clustering for Counties that have spent the most while visiting scotland

...